In [8]:
# chapter 3-2 webページを簡単に取得する

In [1]:
import requests

In [2]:
r = requests.get('https://gihyo.jp/dp')
# webページを取得

In [3]:
type(r)
# get関数の戻り値はレスポンス型

requests.models.Response

In [5]:
r.status_code
# HTTPステータスコードを取得

200

In [6]:
r.headers['content-type']
# HTTPヘッダーの辞書を取得

'text/html; charset=UTF-8'

In [7]:
r.encoding
# HTTPヘッダーから得られたエンコーディングを取得

'UTF-8'

In [ ]:
r.text
# str型にデコードしたレスポンスボディを取得

In [ ]:
r.content
# bytes型のレスポンスボディを取得

In [14]:
r = requests.get('http://weather.livedoor.com/forecast/webservice/json/v1?city=130010')
# 東京の天気をjson形式で取得

In [ ]:
r.json()

In [16]:
r = requests.post('http://httpbin.org/post', data={'key1': 'value1'})
# POSTメソッドで送信

In [19]:
r = requests.get('http://httpbin.org/get',
                headers={'user-agent': 'my-crawler/1.0 (+foo@example.com)'})
# キーワード引数headersにdictで指定してリクエストにHTTPヘッダーを追加

In [22]:
r = requests.get('https://api.github.com/user',
                auth=('discocactus', '<password>'))
# Basic認証のユーザー名とパスワードの組をキーワード引数authで指定

In [24]:
r = requests.get('http://httpbin.org/get', params={'key1': 'value1'})
# URLのパラメーターは引数paramsで指定することも可能

In [34]:
# 複数のページを連続してクロールする場合は、Sessionオブジェクトを使うのが効果的

In [26]:
s = requests.Session()

In [29]:
s.headers.update({'user-agent': 'my-crawler/1.0 (+foo@example.com)'})
# HTTPヘッダーを複数のリクエストで使い回す

In [30]:
r = s.get('https://gihyo.jp/')
# Sessionオブジェクトでもrequestsの様にget(), post()などのメソッドが使える

In [31]:
r = s.get('https://gihyo.jp/dp')

In [33]:
# chapter 3-3 HTMLのスクレイピング

In [59]:
# lxmlによるスクレイピング

In [35]:
import lxml.html

In [36]:
tree = lxml.html.parse('index.html')
# parse()関数でファイルパスを指定してパース
# URLを指定することも可能だが細かい指定ができない

In [37]:
from urllib.request import urlopen

In [38]:
tree = lxml.html.parse(urlopen('http://example.com/'))
# ファイルオブジェクトを指定してパースすることも可能

In [39]:
type(tree)

lxml.etree._ElementTree

In [40]:
html = tree.getroot()

In [41]:
type(html)

lxml.html.HtmlElement

In [42]:
# fromstring()関数で文字列(strまたはbytes)をパースできる
# ただし、encodingが指定されたXML宣言を含むstrをパースすると、ValueErrorが発生するので注意

In [72]:
html = lxml.html.fromstring('''
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id="main">今日のくだもの</h1>
    <ul>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ul>
    </body>
    </html>''')

In [44]:
type(html)

lxml.html.HtmlElement

In [73]:
html.xpath('//li')
# XPathにマッチする要素のリストを取得

[<Element li at 0x10e7d9c28>,
 <Element li at 0x10e7d9c78>,
 <Element li at 0x10e7d9cc8>]

In [74]:
html.cssselect('li')
# CSSセレクターにマッチする要素のリストを取得

[<Element li at 0x10e7d9c28>,
 <Element li at 0x10e7d9c78>,
 <Element li at 0x10e7d9cc8>]

In [49]:
h1 = html.xpath('//h1')[0]

In [50]:
h1.tag
# タグの名前

'h1'

In [51]:
h1.text
# 要素のテキスト

'今日のくだもの'

In [52]:
h1.get('id')
# 属性の値

'main'

In [53]:
h1.attrib
# 全属性を表すdict-likeなオブジェクト

{'id': 'main'}

In [54]:
h1.getparent()
# 親要素

<Element body at 0x10e6bc3b8>

In [57]:
%%writefile scrape_by_lxml.py

import lxml.html

# HTMLファイルを読み込みgetroot()メソッドでHtmlElementオブジェクトを得る
tree = lxml.html.parse('index.html')
html = tree.getroot()

# cssselect()メソッドでa要素のリストを取得して、個々のa要素に対して処理を行う
for a in html.cssselect('a'):
    # href属性とリンクのテキストを取得して表示する
    print(a.get('href'), a.text)

Overwriting scrape_by_lxml.py


In [ ]:
!python scrape_by_lxml.py

In [60]:
# Beautiful Soupによるスクレイピング

In [61]:
from bs4 import BeautifulSoup

In [62]:
with open('index.html') as f:
    soup = BeautifulSoup(f, 'html.parser')
# 第1引数にファイルオブジェクトを指定してBeautifulSoupオブジェクトを生成
# BeautifulSoupにはファイル名やURLを指定することはできない
# 第2引数にパーサーを指定する

In [70]:
# BeautifulSoupのコンストラクターにはHTMLの文字列を渡すことも可能
soup = BeautifulSoup('''
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id="main">今日のくだもの</h1>
    <ul>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ul>
    </body>
    </html>''', 'html.parser')

In [64]:
soup.h1

<h1 id="main">今日のくだもの</h1>

In [65]:
type(soup.h1)

bs4.element.Tag

In [66]:
soup.h1.name

'h1'

In [67]:
soup.h1.string

'今日のくだもの'

In [68]:
type(soup.h1.string)

bs4.element.NavigableString

In [71]:
soup.ul.text

'\nりんご\nみかん\nぶどう\n'

In [75]:
type(soup.h1.text)

str

In [76]:
soup.h1['id']

'main'

In [77]:
soup.h1.get('id')

'main'

In [78]:
soup.h1.attrs

{'id': 'main'}

In [79]:
soup.h1.parent

<body>
<h1 id="main">今日のくだもの</h1>
<ul>
<li>りんご</li>
<li class="featured">みかん</li>
<li>ぶどう</li>
</ul>
</body>

In [80]:
soup.li
# 複数の要素がある場合は先頭の要素が取得される

<li>りんご</li>

In [81]:
soup.find('li')

<li>りんご</li>

In [83]:
soup.find_all('li')
# find_all()メソッドで指定した名前の要素のリストを取得

[<li>りんご</li>, <li class="featured">みかん</li>, <li>ぶどう</li>]

In [84]:
soup.find_all('li', class_='featured')
# キーワード引数でclassなどの属性を指定できる
# classは予約語なのでclass_を使うことに注意

[<li class="featured">みかん</li>]

In [85]:
soup.find_all(id='main')

[<h1 id="main">今日のくだもの</h1>]

In [87]:
soup.find_all('li', class_='featured')

[<li class="featured">みかん</li>]

In [88]:
soup.find_all(id='main')
# タグ名を省略して属性のみで探すことも可能

[<h1 id="main">今日のくだもの</h1>]

In [86]:
soup.select('li')
# select()メソッドでCSSセレクターにマッチする要素を取得

[<li>りんご</li>, <li class="featured">みかん</li>, <li>ぶどう</li>]

In [90]:
soup.select('li.featured')

[<li class="featured">みかん</li>]

In [91]:
soup.select('#main')

[<h1 id="main">今日のくだもの</h1>]

In [92]:
%%writefile scrape_by_bs4.py

from bs4 import BeautifulSoup

with open('index.html') as f:
    soup = BeautifulSoup(f, 'html.parser')

for a in soup.find_all('a'):
    print(a.get('href'), a.text)

Writing scrape_by_bs4.py


In [ ]:
!python scrape_by_bs4.py

In [95]:
# pyqueryによるスクレイピング

In [96]:
from pyquery import PyQuery as pq

In [97]:
d = pq(filename='index.html')

In [98]:
d = pq(url='http://example.com/')

In [99]:
d = pq('''
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id="main">今日のくだもの</h1>
    <ul>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ul>
    </body>
    </html>''')

In [100]:
d('h1')

[<h1#main>]

In [101]:
type(d('h1'))

pyquery.pyquery.PyQuery

In [102]:
d('h1')[0]

<Element h1 at 0x10e654988>

In [103]:
d('h1').text()

'今日のくだもの'

In [104]:
d('h1').attr('id')

'main'

In [105]:
d('h1').attr.id

'main'

In [106]:
d('h1').attr['id']

'main'

In [107]:
d('h1').parent()

[<body>]

In [108]:
d('li')

[<li>, <li.featured>, <li>]

In [109]:
d('li.featured')

[<li.featured>]

In [110]:
d('#main')

[<h1#main>]

In [111]:
d('body').find('li')

[<li>, <li.featured>, <li>]

In [112]:
d('li').filter('.featured')

[<li.featured>]

In [116]:
d('li').eq(1)

[<li.featured>]

In [117]:
# RSSのスクレイピング